<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/r_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# why use R here?  
# edgarWebR pulls sections really well
# do i want to find a python library for the same thing?  sure.
# do i want to write a python utils file for the same thing?  not really but we may have to!

# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
%%R

#R GETTER.  For this example, only one ticker (head,1) is pulled 
#file is saved into local 
#then python code cell pushes file to google sheet

devtools::install_github("mwaldstein/edgarWebR")
#devtools::install_github("tidyverse/googlesheets4")
devtools::install_github("r-lib/xml2") #this for edgarWebR 
#devtools::install_version("xml2", version = "1.2.2", repos = "http://cran.us.r-project.org")
#file creates a set of csv from ticker list which include metadata & text data.

library(edgarWebR) #this is an up to date library with an active maintainer.
library(xml2)
library(knitr)
library(dplyr)
library(purrr)
library(rvest)
library(tidyr)
library(readr)

#library(log4r) TODO logging file.
#library(googlesheets4)
#gs4_deauth()
#tickers list
#str_sheet <- "1_xcDVKjR2jqE-w5LqxIWnrDYstpSrE5nFSSY0WXNOVE"
#df_tickers <- read_sheet(str_sheet)

get_filings_links <-function(str_ticker) {
  df_filings <- company_filings(str_ticker, type = "10-", count = 20)
  df_filings <- df_filings[df_filings$type == "10-K" | df_filings$type == "10-Q", ]
  df_filing_infos <- map_df(df_filings$href, filing_information)
  df_filings <- bind_cols(df_filings, df_filing_infos)
  return(head(as_tibble(df_filings),6))
}

df_tickers <- read_csv('test_ticker_list.csv')

#file creates a set of csv from ticker list which include metadata & text data.
df_tickers <- head(df_tickers,1)

get_section_text <- function(str_href, str_section, str_search) {
  
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)

  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #we could do some text preprocessing here.

  df_txt <- as_tibble(df_txt) %>%
    mutate(section = str_search)

  return(df_txt)
}



R[write to console]: Skipping install of 'edgarWebR' from a github remote, the SHA1 (e7fa70ea) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'xml2' from a github remote, the SHA1 (9928883d) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Parsed with column specification:
cols(
  X1 = col_double(),
  `GICS Sector` = col_character(),
  Symbol = col_character(),
  mkt_cap = col_double()
)



In [11]:
%%R

get_document_text <- function(str_ticker, force = FALSE) { #not using force yet
  start_time <- Sys.time()
  
  print(str_ticker)
  
  str_write_name <- str_ticker 
  #paste0("~/data/",str_ticker,"_sec_text")

  print("get filings links ...")

  df_filings <- get_filings_links(str_ticker)

  #print(df_filings)

  print("get section text ...")

  df_data <- (df_filings) %>% 
    rowwise() %>%
    mutate(nest_discussion = map(.x = href, str_section = 'item 2|item 7',str_search = 'discussion', .f = get_section_text)) %>%
    mutate(nest_qualitative = map(.x = href, str_section = 'item 3|item 7a', str_search = 'qualitative', .f = get_section_text)) %>%
    mutate(nest_controls = map(.x = href, str_section = 'item 4|item 9a',str_search = 'controls', .f = get_section_text)) %>%
    mutate(nest_risk = map(.x = href, str_section = 'item 1',str_search = 'risk factors', .f = get_section_text)) %>%
    ungroup() %>%
    select(period_date,filing_date,type,form_name,documents,nest_discussion,nest_qualitative,nest_controls,nest_risk) %>%
    group_by(period_date) %>%
    arrange(desc(period_date))
  
  #jenky - find a rowwise application
  a <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_discussion) %>%
    unnest(nest_discussion)
  b <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_qualitative) %>%
    unnest(nest_qualitative)
  c <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_controls) %>%
    unnest(nest_controls)
  d <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_risk) %>%
    unnest(nest_risk)

  print("write to local csv  ...")

  #ss <- gs4_create(str_write_name)
  
  #df_data <- rbind(a,b,c,d) %>%
  #  googlesheets4::sheet_write(ss, sheet = "sec_data")
  
  df_data <- rbind(a,b,c,d) %>%
    write_csv(paste0(str_write_name,".csv"))

  end_time <- Sys.time()

  print(end_time - start_time)

  return(df_data)
}

#long run.
df_data <- map_df(df_tickers$Symbol, get_document_text)


[1] "GOOG"
[1] "get filings links ..."
[1] "get section text ..."
[1] "write to local csv  ..."
Time difference of 1.032341 mins


In [0]:

#save single file to google sheets
#to do: loop thru all tickers in dir.

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas as pd

df_data = pd.read_csv('GOOG.csv')

from gspread_dataframe import set_with_dataframe
title = 'sec_data'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df_data) 